## PDF Page Rotation Angle Detection Task

Objective:
Implement the `determine_rotation_angle` function within the given code structure to detect the rotation angle of each page in a PDF file.

Code Structure:
The main function `rotate_all_pages_upright` is already implemented, but if necessary you are allowed to change its implementation. Your task is to complete the `determine_rotation_angle` function.

Input:
- A PDF file path (the function should be able to handle various PDF files)

Output:
- A list of integers, where each integer represents the rotation angle needed for a page in the PDF

Rotation Angle:
- The rotation angle should be in degrees, normalized to the range [0, 359].
- 0 means the page is already upright
- 90 means the page needs to be rotated 90 degrees clockwise to be upright
- and so on...

Task:
1. Implement the `determine_rotation_angle` function:
   - Input: A single page object (PdfReader.PageObject)
   - Output: An integer representing the rotation angle in degrees

2. The function should analyze the content of the page and determine the angle needed to make the page upright.

Requirements:
1. The function should work with different PDF files, not just a specific one.
2. Implement robust methods to determine the correct rotation angle.
3. Handle potential exceptions or edge cases (e.g., pages with mixed orientations, complex layouts).
4. Optimize for both accuracy and processing speed, as the function will be called for each page in the PDF.

Additional Considerations:
- You are allowed to use up to 40GB of GPU VRAM if necessary for your implementation.
- You may create as many additional functions as needed to support your implementation.
- You may use additional libraries if required, but ensure they are imported properly.
- Provide clear comments in your code to explain your rotation detection logic.

Testing:
- Test your implementation with various types of PDFs to ensure its robustness and generalizability.
- The main script provides a way to test your implementation on a file named "grouped_documents.pdf".

Note:
The task involves determining the rotation angle only. The actual rotation of the pages is not required in this implementation.

In [3]:
from typing import List
from PyPDF2 import PdfReader, PdfWriter

import numpy as np
import cv2

from skimage.transform import radon

import time
from PIL import Image
from ImagePreprocessor import ImagePreprocessor

"""
Get the orientation of the text by applying the radon algorithm.

Args: 
    img - opencv image

Returns:
    orientation of the text in degree
"""
def get_rotation_using_radon(img):
    I = img
    if len(img.shape) == 3:
        I = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    h, w = I.shape

    # If the resolution is high, resize the image to reduce processing time.
    if (w > 640):
        I = cv2.resize(I, (640, int((h / w) * 640)))

    # remove black bars from image
    I = ImagePreprocessor().remove_border(I)

    I = I - np.mean(I)  # Demean; make the brightness extend above and below zero
    # Do the radon transform
    sinogram = radon(I)
    # Find the RMS value of each row and find "busiest" rotation,
    # where the transform is lined up perfectly with the alternating dark
    # text and white lines
    r = np.array([np.sqrt(np.mean(np.abs(line) ** 2)) for line in sinogram.transpose()])
    rotation = np.argmax(r)


    # TODO detect if text is upside down
    # https://stackoverflow.com/questions/55654142/detect-if-an-ocr-text-image-is-upside-down

    return 90 - rotation


"""
Determine the rotation angle from a machine readable pdf page.

Args: 
page (PdfReader.PageObject) - Page of a pdf document

Returns: 
the angle by which the pdf page needs to be rotated in order 
for the text to be horizontal
"""
def from_machine_generated_document(page):
    angles = []

    def visitor_text(txt, curr_trans_mat, txt_matrix, font_dict, font_size):
        a, b, c, d, e, f = curr_trans_mat
        # get applied rotation over the z axis
        angle = np.arctan2(c, a)
        nonlocal angles
        angles.append(angle)
        # print(f"txt: [{txt}], ctm: {curr_trans_mat}, tm: {txt_matrix}, angle: {np.arctan2(c,a)}")

    page.extract_text(visitor_text=visitor_text)
    first_non_zero_idx = angles.index(next(filter(lambda x: x != 0, angles), 0))

    rotation = int(angles[first_non_zero_idx] * 100)
    cw_correction_rotation = -1 * rotation if rotation < 0 else 360 - rotation
    return cw_correction_rotation % 360


"""
Read image from the images property of the pdf page.

Args: 
page (PdfReader.PageObject) - Page of a pdf document

Returns: 
list of cv2 images
"""
def read_img_from_images_property(page):
    if len(page.images) > 0:
        images = []
        for image in page.images:

            data = image.data
            nparr = np.frombuffer(data, np.uint8)
            orig = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)

            # the read image seems to be flipped and turned upside down
            orig = cv2.flip(cv2.rotate(orig, cv2.ROTATE_180), 1)
            images.append(orig)
        return images

    return []


"""
Determine the rotation angle from a pdf file containing a scanned image.

Args: 
page (PdfReader.PageObject) - Page of a pdf document

Returns: 
the angle by which the pdf page needs to be rotated in order 
for the text to be horizontal
"""
def from_scanned_document(page):
    images = read_img_from_images_property(page)
    if len(images) == 0:
        images = retrieve_images_from_xobjects(page)
    if len(images) > 0:
        orig = images[0]
        rotation = get_rotation_using_radon(orig)
        cw_correction_rotation = -1 * rotation if rotation < 0 else 360 - rotation
        return cw_correction_rotation % 360
    return 99999999

"""
Retrieve images from xobjects inside the page.

Args: 
page (PdfReader.PageObject) - Page of a pdf document

Returns: 
list of cv2 images 
"""
def retrieve_images_from_xobjects(page):
    images = []

    # read images from xobjects
    resource = page["/Resources"]
    if "/XObject" in resource.keys():
        xobject = resource["/XObject"].get_object()
        for obj in xobject:
            a = xobject[obj]
            if a["/Subtype"] == "/Image":
                mode = "p"
                if a["/ColorSpace"] == '/DeviceRGB':
                    mode = "RGB"
                data = a.get_data()
                # for jpeg images
                if "/DCTDecode" in a["/Filter"]:
                    nparr = np.frombuffer(data, np.uint8)
                    orig = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
                    images.append(orig)
                elif "/FlateDecode" in a["/Filter"]:
                    size = (a['/Width'], a['/Height'])
                    img = Image.frombytes(mode, size, data)
                    tocv2 = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
                    images.append(tocv2)

    return images


"""
Check if there are images contained inside  xobjects

Args: 
page (PdfReader.PageObject) - Page of a pdf document

Returns: 
True if yes false if not
"""
def no_images_inside_xobjects(page):
    return len(retrieve_images_from_xobjects(page)) == 0


"""
Determine the rotation angle in order to rotate the text into an ocr 
parseable position. 

Args: 
page (PdfReader.PageObject) - Page of a pdf document 
"""
def determine_rotation_angle(page):

    if len(page.images) == 0 and no_images_inside_xobjects(page):
        return from_machine_generated_document(page)
    else:
        return from_scanned_document(page)


def rotate_all_pages_upright(input_pdf: str) -> List[int]:
    """
    Analyze all pages in the input PDF and determine the rotation angle needed for each page.

    Args:
    input_pdf (str): The file path of the input PDF.

    Returns:
    List[int]: A list of rotation angles (in degrees) for each page.
               The angles are normalized to be in the range [0, 359].
               0 means no rotation needed, 90 means 90 degrees clockwise, etc.
    """
    reader = PdfReader(input_pdf)
    writer = PdfWriter()

    angles = []
    for page_number in range(len(reader.pages)):
        current_page = reader.pages[page_number]
        rotation_angle = determine_rotation_angle(current_page)
        angles.append(rotation_angle)

    return angles


# Usage
input_pdf: str = "grouped_documents.pdf"
input_pdf: str = "NXP_Arbeitszeugnis_full.pdf"
input_pdf: str = "classification.pdf"
rotation_angles: List[int] = rotate_all_pages_upright(input_pdf)
print(f"Rotation angles for each page: {rotation_angles}")

Rotation angles for each page: [270, 83, 48, 303, 341, 270, 340, 29, 188, 196, 78, 348, 293, 114, 212, 331, 270, 272, 0, 0, 0, 270, 270, 0, 0, 341, 273, 326, 11]
